In [0]:
batting_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/brijeshpatel4547@gmail.com/IPL_data/battingSummary.csv")
bowling_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/brijeshpatel4547@gmail.com/IPL_data/bowlingSummary.csv")
match_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/brijeshpatel4547@gmail.com/IPL_data/matchSummary.csv")
player_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/brijeshpatel4547@gmail.com/IPL_data/players.csv")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [0]:
# +++++++++++++++++++++++++++++++++++++++++++++++++<<< Data Cleaning >>>+++++++++++++++++++++++++++++++++++++++++++++++++

batting_df = batting_df.withColumn('out/not_out',col('out/not_out').cast("Integer")) \
                        .withColumn('battingPos',col('battingPos').cast("Integer")) \
                        .withColumn('runs',col('runs').cast("Integer")) \
                        .withColumn('balls',col('balls').cast("Integer")) \
                        .withColumn('4s',col('4s').cast("Integer")) \
                        .withColumn('6s',col('6s').cast("Integer")) \
                        .withColumn('SR',col('SR').cast("Float"))
batting_df.printSchema()

bowling_df = bowling_df.withColumn('overs',col('overs').cast("Float")) \
                        .withColumn('maiden',col('maiden').cast("Integer")) \
                        .withColumn('runs',col('runs').cast("Integer")) \
                        .withColumn('wickets',col('wickets').cast("Integer")) \
                        .withColumn('economy',col('economy').cast("Integer")) \
                        .withColumn('0s',col('0s').cast("Integer")) \
                        .withColumn('4s',col('4s').cast("Integer")) \
                        .withColumn('6s',col('6s').cast("Integer")) \
                        .withColumn('wides',col('wides').cast("Integer")) \
                        .withColumn('noBalls',col('noBalls').cast("Integer"))
bowling_df.printSchema()
# +++++++++++++++++++++++++++++++++++++++++++++++++<<< Data Cleaning >>>+++++++++++++++++++++++++++++++++++++++++++++++++


root
 |-- match_id: string (nullable = true)
 |-- match: string (nullable = true)
 |-- teamInnings: string (nullable = true)
 |-- battingPos: integer (nullable = true)
 |-- batsmanName: string (nullable = true)
 |-- out/not_out: integer (nullable = true)
 |-- runs: integer (nullable = true)
 |-- balls: integer (nullable = true)
 |-- 4s: integer (nullable = true)
 |-- 6s: integer (nullable = true)
 |-- SR: float (nullable = true)

root
 |-- match_id: string (nullable = true)
 |-- match: string (nullable = true)
 |-- bowlingTeam: string (nullable = true)
 |-- bowlerName: string (nullable = true)
 |-- overs: float (nullable = true)
 |-- maiden: integer (nullable = true)
 |-- runs: integer (nullable = true)
 |-- wickets: integer (nullable = true)
 |-- economy: integer (nullable = true)
 |-- 0s: integer (nullable = true)
 |-- 4s: integer (nullable = true)
 |-- 6s: integer (nullable = true)
 |-- wides: integer (nullable = true)
 |-- noBalls: integer (nullable = true)



IPL DATA ANALYSIS.

Primary Analysis:
 1. Top 10 batsmen based on past 3 years total runs scored.
 2. Top 10 batsmen based on past 3 years batting average. (min 60 balls faced in
 each season)
 3. Top 10 batsmen based on past 3 years strike rate (min 60 balls faced in each
 season)
 4. Top 10 bowlers based on past 3 years total wickets taken.
 5. Top 10 bowlers based on past 3 years bowling average. (min 60 balls bowled in
 each season)
 6. Top 10 bowlers based on past 3 years economy rate. (min 60 balls bowled in
 each season)
 7. Top 5 batsmen based on past 3 years boundary % (fours and sixes).
 8. Top 5 bowlers based on past 3 years dot ball %.
 9. Top 4 teams based on past 3 years winning %.
 10.Top 2 teams with the highest number of wins achieved by chasing targets over
 the past 3 years.


In [0]:
# Top 10 batsmen based on past 3 years total runs scored.
highest_runs = batting_df.groupBy("batsmanName").agg(sum('runs').alias('Total_runs')).orderBy(col('Total_runs').desc()).limit(10)

print("Batsman's with Highest Runs :")
highest_runs.show()


Batsman's with Highest Runs :
+---------------+----------+
|    batsmanName|Total_runs|
+---------------+----------+
|    ShubmanGill|      1851|
|   FafduPlessis|      1831|
| RuturajGaikwad|      1593|
|        KLRahul|      1516|
|     JosButtler|      1509|
|  ShikharDhawan|      1392|
|     ViratKohli|      1385|
|    SanjuSamson|      1304|
|SuryakumarYadav|      1225|
|   GlennMaxwell|      1214|
+---------------+----------+



In [0]:
#Top 10 batsmen based on past 3 years batting average. (min 60 balls faced )
batavg = batting_df.groupBy("batsmanName").agg(sum('runs').alias('Total_runs'), \
                                      sum('balls').alias('Total_balls_faced'), \
                                      sum('out/not_out').alias('Innings_Out'))
batavg = batavg.withColumn("Average", round(batavg.Total_runs/batavg.Innings_Out,3)) \
                .filter(batavg.Total_balls_faced > 60) \
                .orderBy(col("Average").desc()).limit(10)
print("Batsman's with Highest Average :")
batavg.show()


Batsman's with Highest Average :
+---------------+----------+-----------------+-----------+-------+
|    batsmanName|Total_runs|Total_balls_faced|Innings_Out|Average|
+---------------+----------+-----------------+-----------+-------+
|  SaurabhTiwary|       115|               97|          2|   57.5|
|        KLRahul|      1516|             1148|         30| 50.533|
|   CameronGreen|       452|              282|          9| 50.222|
|     RinkuSingh|       648|              434|         13| 49.846|
|HeinrichKlaasen|       448|              253|          9| 49.778|
|    DevonConway|       924|              654|         19| 48.632|
|   SaiSudharsan|       507|              370|         11| 46.091|
|   FafduPlessis|      1831|             1300|         42| 43.595|
|    DavidMiller|       864|              628|         20|   43.2|
|     JosButtler|      1509|             1027|         36| 41.917|
+---------------+----------+-----------------+-----------+-------+



In [0]:
#Top 10 batsmen based on past 3 years strike rate (min 60 balls faced)

batSR = batting_df.groupBy("batsmanName").agg(avg('SR').alias('Strike Rate'), \
                                      sum('balls').alias('Total_balls_faced'))
batSR = batSR.filter(batSR.Total_balls_faced > 60) \
                .orderBy(col("Strike Rate").desc()).drop("Total_balls_faced").limit(10) \
                .withColumn('Strike Rate',round(col("Strike Rate"),2))
print("Batsman's with Highest Strike Rate :")
batSR.show()

Batsman's with Highest Strike Rate :
+---------------+-----------+
|    batsmanName|Strike Rate|
+---------------+-----------+
|HeinrichKlaasen|     182.17|
|   JiteshSharma|     156.15|
|  JaydevUnadkat|     150.62|
|   GlennMaxwell|      150.5|
|     DhruvJurel|      146.3|
|       TimDavid|     145.32|
|  SaurabhTiwary|     144.99|
|   PrerakMankad|     144.44|
| RavindraJadeja|      143.5|
|     RinkuSingh|     143.19|
+---------------+-----------+



In [0]:
bowling_df.printSchema()

root
 |-- match_id: string (nullable = true)
 |-- match: string (nullable = true)
 |-- bowlingTeam: string (nullable = true)
 |-- bowlerName: string (nullable = true)
 |-- overs: float (nullable = true)
 |-- maiden: integer (nullable = true)
 |-- runs: integer (nullable = true)
 |-- wickets: integer (nullable = true)
 |-- economy: integer (nullable = true)
 |-- 0s: integer (nullable = true)
 |-- 4s: integer (nullable = true)
 |-- 6s: integer (nullable = true)
 |-- wides: integer (nullable = true)
 |-- noBalls: integer (nullable = true)



In [0]:
# Top 10 bowlers based on past 3 years total wickets taken.
total_wickets = bowling_df.groupBy("bowlerName").agg(sum('wickets').alias("Total Wickets")) \
                          .orderBy(col('Total Wickets').desc()).limit(10)

total_wickets.show()

+-----------------+-------------+
|       bowlerName|Total Wickets|
+-----------------+-------------+
|    MohammedShami|           67|
|  YuzvendraChahal|           66|
|     HarshalPatel|           65|
|       RashidKhan|           63|
|        AveshKhan|           47|
|    ArshdeepSingh|           45|
|     KagisoRabada|           45|
|VarunChakravarthy|           44|
|    ShardulThakur|           43|
|       TrentBoult|           42|
+-----------------+-------------+



In [0]:
# Top 10 bowlers based on past 3 years bowling average. (min 60 balls bowled)
bowlingAvg = bowling_df.groupBy("bowlerName").agg(
    sum('runs').alias("Total_Runs_Conceeded"), \
    sum('wickets').alias("Total_Wickets_Taken"),
    sum('overs').alias("Total_Overs"))

bowlingAvg = bowlingAvg.filter(bowlingAvg.Total_Overs > 10).filter(bowlingAvg.Total_Wickets_Taken > 1)

bowlingAvg = bowlingAvg.withColumn("Bowling_Average", round(col('Total_Runs_Conceeded')/col('Total_Wickets_Taken'),2)) \
                        .orderBy(col("Bowling_Average").asc()).limit(10).drop("Total_Overs")
bowlingAvg.show()


+----------------+--------------------+-------------------+---------------+
|      bowlerName|Total_Runs_Conceeded|Total_Wickets_Taken|Bowling_Average|
+----------------+--------------------+-------------------+---------------+
|        MarkWood|                 130|                 11|          11.82|
|     MohitSharma|                 361|                 27|          13.37|
|    AkashMadhwal|                 219|                 14|          15.64|
|MichaelBracewell|                  95|                  6|          15.83|
|     ChrisWoakes|                  82|                  5|           16.4|
|   MitchellMarsh|                 272|                 16|           17.0|
|      MohsinKhan|                 304|                 17|          17.88|
|    AndreRussell|                 638|                 35|          18.23|
|     DwayneBravo|                 562|                 30|          18.73|
|      AmitMishra|                 227|                 12|          18.92|
+-----------

In [0]:
# Top 10 bowlers based on past 3 years economy rate. (min 60 balls bowled)
bowlingEr = bowling_df.groupBy('bowlerName').agg(
    sum('overs').alias("Total_Overs"),round(avg('economy'),2).alias("Average Economy Rate"))
bowlingEr = bowlingEr.filter(bowlingEr.Total_Overs > 10)
bowlingEr = bowlingEr.orderBy(col("Average Economy Rate")).limit(10)
bowlingEr.show()

+---------------+------------------+--------------------+
|     bowlerName|       Total_Overs|Average Economy Rate|
+---------------+------------------+--------------------+
|    DavidWilley|              26.0|                6.38|
|    SunilNarine|             160.0|                6.38|
|MitchellSantner|              31.0|                6.56|
|      AxarPatel|             132.0|                6.86|
|     RashidKhan|             182.5|                6.87|
|  ShakibAlHasan|              26.0|                6.88|
|  JaspritBumrah|108.20000004768372|                 7.0|
|     SaiKishore|              16.0|                 7.0|
|    ChrisWoakes|              11.0|                 7.0|
|  Naveen-ul-Haq|              28.0|                7.29|
+---------------+------------------+--------------------+



In [0]:
bowling_df = bowling_df.withColumn("Balls_Bowled",(bowling_df.overs % 1) * 10 + floor(bowling_df.overs) * 6) \
                        .withColumn("Balls_Bowled",col("Balls_Bowled").cast('Integer'))
bowling_df.show(2)

+--------+------------------+-----------+-------------+-----+------+----+-------+-------+---+---+---+-----+-------+------------+
|match_id|             match|bowlingTeam|   bowlerName|overs|maiden|runs|wickets|economy| 0s| 4s| 6s|wides|noBalls|Balls_Bowled|
+--------+------------------+-----------+-------------+-----+------+----+-------+-------+---+---+---+-----+-------+------------+
| T203817|Super Kings Vs KKR|        KKR|ShakibAlHasan|  3.0|     0|  33|      0|     11|  6|  2|  3|    0|      0|          18|
| T203817|Super Kings Vs KKR|        KKR|   ShivamMavi|  4.0|     0|  32|      1|      8|  8|  1|  2|    0|      0|          24|
+--------+------------------+-----------+-------------+-----+------+----+-------+-------+---+---+---+-----+-------+------------+
only showing top 2 rows



In [0]:
bowlingDot = bowling_df.groupBy("bowlerName").agg(sum("Balls_Bowled").alias("Total_Balls_Bowled"),\
                                                  sum("0s").alias("Total_Dots"))

bowlingDot = bowlingDot.withColumn("Dot%",round((col("Total_Dots")/col("Total_Balls_Bowled"))*100,2)) \
                        .orderBy(col("Dot%").desc(),col("Total_Dots").desc()).limit(10)
                                   
bowlingDot.show()

+---------------+------------------+----------+-----+
|     bowlerName|Total_Balls_Bowled|Total_Dots| Dot%|
+---------------+------------------+----------+-----+
|    ShreyasIyer|                 6|         4|66.67|
|     ImranTahir|                24|        14|58.33|
|    ReeceTopley|                12|         7|58.33|
| SimarjeetSingh|               108|        57|52.78|
|   YudhvirSingh|                48|        25|52.08|
|MoisesHenriques|                60|        31|51.67|
|     MohsinKhan|               258|       130|50.39|
|       MarkWood|                96|        48| 50.0|
|    ChrisWoakes|                66|        33| 50.0|
|    SanjayYadav|                12|         6| 50.0|
+---------------+------------------+----------+-----+



In [0]:
# Total Matches Played by each team

matches_played = match_df.groupBy("team1").agg(count(col("team1")).alias("Team1_Played")).union(match_df.groupBy("team2").agg(count(col("team2")).alias("Team2_Played")))

matches_played = matches_played.groupBy("team1").agg(sum("Team1_Played").alias("Matches_Played")) \
                                .withColumnRenamed("team1","Team").orderBy("Matches_Played",ascending = False)

# Total Matches Won
matches_won = match_df.groupBy("winner").agg(count(col("winner")).alias("Total_Wins"))

# Win % in Past 3 Years
matches_joined = matches_played.join(matches_won,matches_played.Team == matches_won.winner,"inner").select("Team","Matches_Played","Total_Wins")

matches_joined = matches_joined.withColumn("Win%",round((col("Total_Wins")/col("Matches_Played"))*100,2)) \
                                .orderBy("Win%",ascending = False)

matches_joined.show(4)


+----+--------------+----------+-----+
|Team|Matches_Played|Total_Wins| Win%|
+----+--------------+----------+-----+
|  GT|            33|        23| 69.7|
| LSG|            29|        17|58.62|
| CSK|            45|        25|55.56|
| RCB|            45|        25|55.56|
+----+--------------+----------+-----+
only showing top 4 rows



In [0]:
matches_chased = match_df.filter(col("margin").like('%wickets')).groupBy("winner").agg(count("Winner").alias("Matches_Won"))

matches_joined_2 = matches_played.join(matches_chased,matches_played.Team == matches_chased.winner,"inner").select("Team","Matches_Played","Matches_Won")
matches_joined_2 = matches_joined_2.withColumn("Win%",round((col("Matches_Won")/col("Matches_Played"))*100,2)) \
                                .orderBy("Win%",ascending = False)

matches_joined_2.show(2)



+----+--------------+-----------+-----+
|Team|Matches_Played|Matches_Won| Win%|
+----+--------------+-----------+-----+
|  GT|            33|         14|42.42|
|  DC|            43|         14|32.56|
+----+--------------+-----------+-----+
only showing top 2 rows



In [0]:
batting_df.printSchema()

root
 |-- match_id: string (nullable = true)
 |-- match: string (nullable = true)
 |-- teamInnings: string (nullable = true)
 |-- battingPos: integer (nullable = true)
 |-- batsmanName: string (nullable = true)
 |-- out/not_out: integer (nullable = true)
 |-- runs: integer (nullable = true)
 |-- balls: integer (nullable = true)
 |-- 4s: integer (nullable = true)
 |-- 6s: integer (nullable = true)
 |-- SR: float (nullable = true)



In [0]:
runs_df = batting_df.groupBy("batsmanName").agg(sum('runs').alias("TotalRuns"),\
                                                sum('4s').alias("Total4s"),\
                                                sum('6s').alias("Total6s"),\
                                                sum('balls').alias("BallFaced"))

runs_df = runs_df.withColumn('Boundary_runs',(col("Total4s")*4 + col("Total6s")*6))\
                    .withColumn('Boundary%',round(col("Boundary_runs")/col("TotalRuns") * 100,2))\
                    .filter(runs_df.BallFaced > 60)\
                    .drop("Total4s","Total6s")

runs_df = runs_df.orderBy("Boundary%",ascending=False).limit(5)

runs_df.show()

+------------+---------+---------+-------------+---------+
| batsmanName|TotalRuns|BallFaced|Boundary_runs|Boundary%|
+------------+---------+---------+-------------+---------+
|  PatCummins|      156|       80|          124|    79.49|
|DewaldBrevis|      161|      113|          122|    75.78|
|AndreRussell|      745|      468|          564|     75.7|
|  KyleMayers|      365|      246|          276|    75.62|
| SunilNarine|      154|      112|          116|    75.32|
+------------+---------+---------+-------------+---------+

